In [82]:
def projection_matrix_1d(b):
    """Compute the projection matrix onto the space spanned by `b`
    Args:
        b: ndarray of dimension (D,), the basis for the subspace
    
    Returns:
        P: the projection matrix
    """
    P = np.dot(b, b.T) / np.dot(b.T, b) # b是列向量
    return P

In [83]:
def neutralize(x, b):
    """Compute the perpendicular vector onto the space spanned by `b`
    Args:
        x: the vector to be projected
        b: ndarray of dimension (D,), the basis for the subspace
    
    Returns:
        P: projection of x in space spanned by b
    """
    b = b[1] - b[0]
    x = x.reshape((-1, 1))
    b = b.reshape((-1, 1))
    # x, b都是列向量
    P = np.dot(projection_matrix_1d(b), x)
    P = x - P
    return P

In [84]:
def findStrInFile(str):
    f_pro = open('professions.txt')
    for line in f_pro.readlines():
        l = len(line)
        if str == line[:l - 1]:
            f_pro.close()
            return True
    f_pro.close()
    return False

In [89]:
import string
import re
from gensim.models import word2vec
import numpy as np

# 对于 model 中训练得到的词向量，依次查看词是否为职业词，若是，则做中性化处理

fout = open('wiki.zh.debias.txt', 'w')
model = word2vec.Word2Vec.load('wiki.zh.model')

#W = ["他", "她", "他们", "她们", "男性", "女性", "父亲", "母亲", "儿子", "女儿", 
#     "爸爸", "妈妈", "男孩", "女孩", "男生", "女生", "男人", "女人", "男子", "女子"]
D = list()
D.append(model.wv["他"])
D.append(model.wv["她"])

D = np.array(D)
#B = generate_subspace(D, 10)

for word in model.wv.vocab.keys():
    if findStrInFile(word):
        w = model.wv[word]
        w = np.array(w)
        w_n = neutralize(w, D)
        s = ""
        s += word + " "
        for i in w_n:
            s += str(i[0]) + " "
        s += "\n"
        fout.write(s)
    else:
        w = model.wv[word]
        s = ""
        s += word + " "
        for i in w:
            s += str(i) + " "
        s += "\n"
        fout.write(s)

fout.close()